https://github.com/ftsrg/chc2c
extraction



In [ ]:
# diff

# diff




In [ ]:
from kdrag.all import *

def diff(e, x):
    if e.eq(x):
        return 1
    elif smt.is_value(e):
        return 0
    elif smt.is_add(e):
        return smt.Sum([diff(c,x) for c in e.children()])
    elif e.decl() == real.sin:
        return real.cos(e) * diff(e, x)
    elif e.decl() == real.cos:
        return -real.sin(e) * diff(e, x)
    elif e.decl() == real.exp:
        return real.exp(e) * diff(e, x)
    
    


# knuckelproblems


In [6]:


from kdrag.all import *
# Knuckledragger support algebraic datatypes and induction
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

"""
add = kd.define("add", [n,m], 
    n.match_(
        (Nat.Zero, m),
        (Nat.Succ(n), Nat.Succ(add(n, m)))
))
add.defn
"""
add.defn


|- ForAll([n, m],
       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!6)))

In [9]:
import kdrag.reflect as reflect

@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# Type resgistry

Some 

add_assoc
add_comm
mul_assoc
mul_comm


COuld reaplce sortdispatch


In [ ]:
# hash cons the types so that we can tag useful stuff on them
types = {}

def DeclareSort(name):
    if name not in types:
        types[name] = type(name, (object,), {})
    return types[name]


# Schematic Vars
Explicitly pulling schematic variables out, may let me refer to them in subproofs (How?)

   G |- A
------------- weaken
     G, v fresh |- A
Well that's annoying.

app(forall x, A, t)




forall x, A  

Maybe I should consider this as A Proof extension rather than proof replacement, likke refinement types


```
Gam |- t   x not in t
-----------
Gam, x |- t 

```


If I merged forall intro and removal


This is just |- forall x, P(x) but with some extra junk.

capture avoiding

https://ncatlab.org/nlab/show/substitution

https://en.wikipedia.org/wiki/Admissible_rule
https://cstheory.stackexchange.com/questions/54600/admissible-rules-in-dependent-type-theory

Gam |- A then sigma Gam |- sigma A


The point was that

prove(t, )

def cprove(vs, p, by=)


a curried form of backward proof tracking.
modus for leaves
comp for moves.
but yeah, split
A /\ B
(A => B => A /\ B)
(A => A \/ B)
(B => A \/ B)

A => B => C

but then aplpying move in context is annoying. Hmm.
And I don't give a shit about anything except quantifier moves so it's wasteful effort.

Maybe the callback method is easier. But it'll have so many expensive calls


def intros():
   




In [ ]:
def modus(ab, a):
    assert smt.is_implies(ab) and ab.arg(0).eq(a)
    assert isinstance(ab, Proof) and isinstance(a, Proof)
    return Proof(ab.arg(1), reasons=[ab, a])

def impl_refl(a):
    return Proof(smt.Implies(a,a))

def comp(ab, bc): # cut sort of
    assert smt.is_implies(ab) and smt.is_implies(bc) and ab.arg(1).eq(bc.arg(0))
    assert isinstance(ab, Proof) and isinstance(bc, Proof)
    return Proof(smt.Implies(ab.arg(0), bc.arg(1)), reasons=[ab, bc])


class Lemma():
    self.backward_proof = impl_refl(goal)
    self.topgoal = goal
    self.curgoal = goal
    

def qed():
    assert self.backwardproof.thm.eq(self.topgoal)
    return self.backwardproof
def fixes():
    herb(self.topgoal)
    self.backwardproof = comp(  , self.backwardproof)(modus(ab,a)))
def prove():
    p = self.prove(curgoal)
    self.backwardproof = modus(self.backwardproof, self.proof)
def intros():
    self.curgoal = self.curgoal.arg(0)
    # goal is unchanged?
    # (a -> b) -> topgoal. uhhh. G -> 
    # 
    # /\ (forall vs, hyps => conc) -> topgoal
    # (hyp1 => conc1) => (hyp2 => conc2) => hyp3 => topgoal
    # goal1 => goal2 => topgoal



Goal
  def to_expr
   

                                                


In [ ]:
def deinstan(pf):
    # get substution if last thing was an instan?
    # Super weird idea.

def cprove(
    vs : list[smt.ExprRef],
    thm: smt.BoolRef,
    by = lambda *args: [] #Callable[smt.ExprRefglist[kd.Proof],
    **kwargs
) -> kd.kernel.Proof:
    """Contextual proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(vs, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)



def indprove(v, thm):
    """Inductive proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(v, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)

def indprove(v : smt.ExprRef, thm, by=None, using=None):
    l = kd.Lemma(smt.ForAll(v, thm))
    v1 = l.fix()
    l.induct(v1, using=using)
    l.auto(by=by)
    return l.qed()

In [ ]:
def subst(vs, p : kd.Proof, subst):
    assert isinstance(p, kd.Proof)
    vs1, body = kd.utils.open_binder_unhygienic(p.fm)
    assert kd.utils.free_in(vs - vs1, body)

# vs

# ok so we can achieve of substutiton by combining herb and instan.
def subst(vs, p : kd.Proof, subst):
    goal = smt.ForAll(vs, smt.substitute(p.fm, *subst.items()))
    l = kd.Lemma(goal)
    vs = l.fixes()
    l.qed(by=p(subst[v] for v in vs))
    #vs, pf = kd.kernel.herb(p)



In [ ]:
class Proof():
    ctx : dict[smt.SortRef, int] # How many de bruijn indices are allowed?
    fm : smt.BoolRef
    

In [ ]:
class Proof():
    vs : list[smt.ExprRef] # ctx
    fm : smt.BoolRef | smt.QuantifierRef
    reasons : list[object]
    def __call__(self, *args, vs=[]):
        smt.substitute(self.fm, *zip(self.vs, args))

def free(vs, fm):
    #fm2 = smt.substitute(fm, *[(v, smt.FreshConst(v.sort())) for v in vs])
    return smt.Lambda(vs, fm).body().eq(fm)
    #return fm2.eq(fm)

def lift(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    return Proof(
        vs = [],
        fm = p.fm,
        reasons = [p]
    )
def lift_forall(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    return Proof(
        vs = vs,
        fm = fm,
        reasons = [p]
    )
def lift_free(ctx, p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    assert all(v in ctx for v in p.vs)
    assert free(ctx, p.fm)
    return Proof(
        vs = ctx,
        fm = p.fm,
        reasons = [p]
    )

"""
  Gam |- forall x, P
---------------
 Gam, x |- P

 This is reversed intro rule. Feels odd.
"""
def open(p : Proof) -> Proof:
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    Proof(p.vs + vs, fm, [p])

def intro(n, p : Proof):
    return Proof(p.vs[n:], smt.ForAll(p.vs[:n], p.fm), [p])

def extend(vs, p : Proof) -> Proof:
    s = smt.Solver()
    s.add(smt.ForAll(p.vs, p.fm))
    s.add(smt.Not(smt.ForAll(p.vs + vs, p.fm)))
    res = s.check()
    if res == smt.unsat:
        return Proof(
            vs = p.vs + vs,
            fm = p.fm,
            reasons = [p]
        )
    else:
        raise Exception("Unknown result from solver")


def subst(p : Proof, ctx : list[smt.ExprRef], subst : dict[smt.ExprRef, smt.ExprRef]):
    assert isinstance(p, Proof)
    assert all(v in p.vs for v in  subst.keys())
    assert free(ctx - p.ctx, p.fm)
    #fm = smt.substitute(p.fm, *[(v, smt.FreshConst(v.sort())) for v in ctx])
    fm = smt.substitute(p.fm, *subst.items())
    # I need to check that ctx2 does not appear in fm. Otherwise I could |- even(zero) ---> zero |- even(zero)
    return Proof(
        vs = ctx,
        fm = fm,
        reasons = [p]
    )


"""
G |- p1    G |- p2 ... 
-------
    G |- p

"""

def prove(ctx, expr, by=None):
    allctx = set(ctx)
    for p in by:
        assert isinstance(p, Proof)
        assert p.ctx == ctx
    s.add(smt.Not(smt.ForAll([ctx], smt.Implies(smt.And(hyp.fm for hyp in by), expr))))
    s.check()




Ok so yes, any forall infecting the lemmas is a problem.
And some things get foralls because its hard an annoying 

qed should 


In [6]:
from kdrag.all import *
import kdrag.theories.real as real
x,y = smt.Reals("x y")
cos = real.cos
sin = real.sin
#os_diff = kd.prove(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)), [real.cos_add, real.cos_neg, real.sin_neg])

_l = kd.Lemma(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)))
_x, _y = _l.fixes()
_l.symm()
_l.eq(cos(_x + (-_y)))
#_l.rw(kd.prove(cos(_x - _y) == cos(_x + (-_y))))
#_l.rw(kd.prove(smt.ForAll([x,y], x - y == (x + (-y))))(_x, _y))
_l.rw(real.cos_add(_x, -_y))
#_l.auto(by=[real.cos_add, real.sin_add])
_l.auto(by=[real.cos_neg(_y), real.sin_neg(_y)])
_l.lemmas
_l.qed()
cos_diff = _l.lemmas


# Homomorphism theorems


In [ ]:
from kdrag.all import *
import kdrag.property as prop
import kdrag.theories.algebra.group as group

A = smt.DeclareSort('A')
def Group(A):
    mul = smt.Function('mul', A, A, A)
    inv = smt.Function('inv', A, A)
    id_ = smt.Const('id', A)
    x,y,z = smt.Consts('x y z', A)
    kd.notation.mul.register(A, mul)
    kd.notation.invert.register(A, inv)
    assoc = kd.axiom(smt.ForAll([x,y,z], x * (y * z) == (x * y) * z))
    id_left = kd.axiom(smt.ForAll([x], id_ * x == x))
    inv_left = kd.axiom(smt.ForAll([x], inv(x) * x == id_))
    return id_, assoc, id_left, inv_left


B = smt.DeclareSort('B')
group.Group.register(A)
idA, assocA, id_leftA, inv_leftA = Group(A)
idB, assocB, id_leftB, inv_leftB = Group(B)

h = smt.Function('h', A, B)
def homo(id_, h):
    A = h.domain(0)
    B = h.range()
    x,y,z = smt.Consts("x y z", A)
    mul = smt.ForAll([x,y], h(x * y) == h(x) * h(y))
    id_ = h(idA) == idB
    inv = smt.ForAll([x], h(~x) == ~h(x))
    return mul, id_, inv
homo(h)





ValueError: ('Not registered in typeclass', (A,))

Universe of groups.

I was saying construct it as permuations  Int -> Int

Nontrivial
But we could also driectly axiomatize

* is composition
id is  x -> x
homomorphisms need to... be careful here. Only push homo through on particular Group?
SetGrp 





In [ ]:

Grp = smt.DeclareSort("Grp")
SetGrp = set_.SetSort(Grp)
inv = smt.Function("inv", Grp, Grp)
# inv = smt.Function("inv", GrpSet, GrpSet)
mul = smt.Function("mul", Grp, Grp) # We just leave mul un

Closed = smt.QForAll([x,y, S], S[x], S[y], S[x * y], S[inv(x)])

smt.QForAll([S], Closed[S], S[x], S[y], S[z],  x*(y*z) == (x * y) * z) # Is there a model where mul can be associative and always defined? 
smt.QForAll([S], Closed[S], S[x], inv(x) * x == x)  # maybe unconditionally actually
smt.QForAll([S], id_ * x == x) # also unconditiojnality